# VM-21 Reserve Calculation: CTE70 Deep Dive

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brandonmbehring-dev/insurance-ai-toolkit/blob/main/notebooks/01_reserves_vm21.ipynb)

This notebook demonstrates the **Conditional Tail Expectation (CTE70)** reserve calculation for Variable Annuities with Guaranteed Lifetime Withdrawal Benefits (GLWB), as required by **NAIC VM-21** regulation.

## Learning Objectives

1. Understand CTE (Conditional Tail Expectation) vs VaR
2. Implement Monte Carlo simulation for reserve estimation
3. Analyze convergence behavior
4. Explore sensitivity to key parameters

---

## 1. Background: VM-21 and CTE70

### What is VM-21?

**VM-21** (Valuation Manual Section 21) is the NAIC's standard for reserving variable annuities with living benefits. It requires insurers to hold reserves based on the **CTE70** (70th percentile Conditional Tail Expectation).

### CTE vs VaR

| Metric | Definition | For CTE70 |
|--------|------------|------------|
| **VaR(70)** | Value at 70th percentile | Single point estimate |
| **CTE70** | Average of worst 30% of outcomes | Captures tail risk better |

$$\text{CTE}_\alpha = \mathbb{E}[X | X > \text{VaR}_\alpha]$$

CTE70 is always ≥ VaR70 because it averages the tail, not just the boundary.

### Why CTE70 for Insurance?

- **Tail risk matters**: Guarantees pay out in adverse scenarios
- **Coherent risk measure**: Unlike VaR, CTE is subadditive
- **Regulatory standard**: VM-21 mandates CTE70 for VA reserves

## 2. Setup

In [ ]:
# Install dependencies (for Colab)
# !pip install numpy pandas matplotlib seaborn --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
np.random.seed(42)  # Reproducibility

print("Setup complete!")

## 3. Policy Parameters

We'll model a typical VA+GLWB policy:

In [ ]:
# Policy parameters
ACCOUNT_VALUE = 350_000      # Initial account value ($)
BENEFIT_BASE = 350_000       # GLWB benefit base ($)
WITHDRAWAL_RATE = 0.05       # 5% annual withdrawal rate
RIDER_FEE = 0.01             # 1% annual rider fee
FUND_FEE = 0.015             # 1.5% annual fund management fee

# Market parameters
RISK_FREE_RATE = 0.04        # 4% risk-free rate
EQUITY_VOL = 0.18            # 18% annual volatility
EQUITY_DRIFT = 0.07          # 7% expected return (before fees)

# Simulation parameters
N_SCENARIOS = 1000           # Number of Monte Carlo paths
N_YEARS = 20                 # Projection horizon
STEPS_PER_YEAR = 12          # Monthly time steps

print(f"Policy: ${ACCOUNT_VALUE:,} account, {WITHDRAWAL_RATE:.0%} withdrawal rate")
print(f"Market: {EQUITY_VOL:.0%} vol, {RISK_FREE_RATE:.0%} discount rate")
print(f"Simulation: {N_SCENARIOS} scenarios × {N_YEARS} years")

## 4. Monte Carlo Simulation Engine

### Equity Path Generation (GBM)

We use Geometric Brownian Motion to simulate equity returns:

$$S_t = S_0 \exp\left[(\mu - \frac{\sigma^2}{2})t + \sigma W_t\right]$$

In [ ]:
def generate_equity_paths(
    n_scenarios: int,
    n_years: int,
    steps_per_year: int,
    drift: float,
    volatility: float,
    initial_value: float = 1.0,
) -> np.ndarray:
    """
    Generate Monte Carlo equity paths using GBM.
    
    Returns:
        Array of shape (n_scenarios, n_steps + 1) with equity multipliers
    """
    dt = 1.0 / steps_per_year
    n_steps = n_years * steps_per_year
    
    # Generate random returns
    z = np.random.standard_normal((n_scenarios, n_steps))
    
    # GBM formula
    log_returns = (drift - 0.5 * volatility**2) * dt + volatility * np.sqrt(dt) * z
    
    # Cumulative product to get price paths
    paths = np.zeros((n_scenarios, n_steps + 1))
    paths[:, 0] = initial_value
    paths[:, 1:] = initial_value * np.exp(np.cumsum(log_returns, axis=1))
    
    return paths

# Generate paths
equity_paths = generate_equity_paths(
    N_SCENARIOS, N_YEARS, STEPS_PER_YEAR, EQUITY_DRIFT, EQUITY_VOL
)

print(f"Generated {equity_paths.shape[0]} paths with {equity_paths.shape[1]} time steps")
print(f"Mean terminal value: {equity_paths[:, -1].mean():.3f}")
print(f"Median terminal value: {np.median(equity_paths[:, -1]):.3f}")

In [ ]:
# Visualize sample paths
fig, ax = plt.subplots(figsize=(12, 6))

time_axis = np.arange(equity_paths.shape[1]) / STEPS_PER_YEAR

# Plot percentile bands
p5 = np.percentile(equity_paths, 5, axis=0)
p25 = np.percentile(equity_paths, 25, axis=0)
p50 = np.percentile(equity_paths, 50, axis=0)
p75 = np.percentile(equity_paths, 75, axis=0)
p95 = np.percentile(equity_paths, 95, axis=0)

ax.fill_between(time_axis, p5, p95, alpha=0.2, label='5th-95th percentile')
ax.fill_between(time_axis, p25, p75, alpha=0.3, label='25th-75th percentile')
ax.plot(time_axis, p50, 'b-', linewidth=2, label='Median')

# Plot a few sample paths
for i in range(5):
    ax.plot(time_axis, equity_paths[i], alpha=0.5, linewidth=0.5)

ax.axhline(y=1.0, color='gray', linestyle='--', label='Initial value')
ax.set_xlabel('Years')
ax.set_ylabel('Equity Multiplier')
ax.set_title('Monte Carlo Equity Paths (GBM)')
ax.legend()
plt.tight_layout()
plt.show()

## 5. GLWB Liability Projection

For each equity path, we project the account value and calculate the present value of guarantee shortfalls.

In [ ]:
def project_glwb_liability(
    equity_paths: np.ndarray,
    account_value: float,
    benefit_base: float,
    withdrawal_rate: float,
    rider_fee: float,
    fund_fee: float,
    risk_free_rate: float,
    steps_per_year: int,
) -> np.ndarray:
    """
    Project GLWB liabilities for each Monte Carlo scenario.
    
    Returns:
        Array of present values of guarantee costs for each scenario
    """
    n_scenarios, n_steps = equity_paths.shape
    n_steps -= 1  # Exclude initial value
    dt = 1.0 / steps_per_year
    
    # Initialize arrays
    av = np.full(n_scenarios, account_value)  # Account value
    bb = np.full(n_scenarios, benefit_base)   # Benefit base
    pv_costs = np.zeros(n_scenarios)          # PV of guarantee costs
    
    annual_withdrawal = benefit_base * withdrawal_rate
    monthly_withdrawal = annual_withdrawal / steps_per_year
    
    for t in range(1, n_steps + 1):
        # Get equity return for this period
        equity_return = equity_paths[:, t] / equity_paths[:, t-1]
        
        # Apply equity growth (net of fees)
        net_return = equity_return * np.exp(-(rider_fee + fund_fee) * dt)
        av = av * net_return
        
        # Process withdrawal
        withdrawal = np.minimum(av, monthly_withdrawal)
        shortfall = np.maximum(0, monthly_withdrawal - av)
        av = np.maximum(0, av - monthly_withdrawal)
        
        # Discount shortfall to present value
        discount_factor = np.exp(-risk_free_rate * t * dt)
        pv_costs += shortfall * discount_factor
    
    return pv_costs

# Calculate liabilities for all scenarios
pv_liabilities = project_glwb_liability(
    equity_paths,
    ACCOUNT_VALUE,
    BENEFIT_BASE,
    WITHDRAWAL_RATE,
    RIDER_FEE,
    FUND_FEE,
    RISK_FREE_RATE,
    STEPS_PER_YEAR,
)

print(f"PV of guarantee costs:")
print(f"  Mean: ${pv_liabilities.mean():,.0f}")
print(f"  Median: ${np.median(pv_liabilities):,.0f}")
print(f"  Max: ${pv_liabilities.max():,.0f}")
print(f"  Scenarios with positive cost: {(pv_liabilities > 0).sum()} ({(pv_liabilities > 0).mean():.1%})")

## 6. CTE70 Calculation

Now we calculate CTE70 - the average of the worst 30% of outcomes.

In [ ]:
def calculate_cte(values: np.ndarray, alpha: float) -> float:
    """
    Calculate Conditional Tail Expectation at level alpha.
    
    CTE_alpha = E[X | X > VaR_alpha]
    
    Args:
        values: Array of scenario values (higher = worse)
        alpha: Percentile level (e.g., 0.70 for CTE70)
    
    Returns:
        CTE at specified level
    """
    sorted_values = np.sort(values)
    cutoff_idx = int(len(values) * alpha)
    tail_values = sorted_values[cutoff_idx:]
    return tail_values.mean()

# Calculate key metrics
var70 = np.percentile(pv_liabilities, 70)
cte70 = calculate_cte(pv_liabilities, 0.70)
cte90 = calculate_cte(pv_liabilities, 0.90)
mean_liability = pv_liabilities.mean()

print("Reserve Metrics:")
print(f"  Mean Liability:  ${mean_liability:>12,.0f}")
print(f"  VaR(70):         ${var70:>12,.0f}")
print(f"  CTE70 (Reserve): ${cte70:>12,.0f}  ← VM-21 Reserve")
print(f"  CTE90:           ${cte90:>12,.0f}")
print(f"")
print(f"CTE70 / Mean ratio: {cte70/mean_liability:.2f}x")
print(f"CTE70 as % of AV:   {cte70/ACCOUNT_VALUE:.1%}")

In [ ]:
# Visualize the distribution and CTE
fig, ax = plt.subplots(figsize=(12, 6))

# Histogram
n, bins, patches = ax.hist(pv_liabilities, bins=50, density=True, alpha=0.7, color='steelblue')

# Color the tail
for i, patch in enumerate(patches):
    if bins[i] >= var70:
        patch.set_facecolor('red')
        patch.set_alpha(0.8)

# Mark key values
ax.axvline(mean_liability, color='green', linestyle='--', linewidth=2, label=f'Mean: ${mean_liability:,.0f}')
ax.axvline(var70, color='orange', linestyle='--', linewidth=2, label=f'VaR70: ${var70:,.0f}')
ax.axvline(cte70, color='red', linestyle='-', linewidth=3, label=f'CTE70: ${cte70:,.0f}')

ax.set_xlabel('PV of Guarantee Cost ($)')
ax.set_ylabel('Density')
ax.set_title('Distribution of GLWB Guarantee Costs\n(Red shading = worst 30% scenarios used for CTE70)')
ax.legend()
plt.tight_layout()
plt.show()

## 7. Convergence Analysis

How many scenarios do we need for a stable CTE70 estimate?

In [ ]:
def convergence_analysis(pv_liabilities: np.ndarray, scenario_counts: List[int]) -> pd.DataFrame:
    """
    Analyze CTE70 convergence as number of scenarios increases.
    """
    results = []
    
    for n in scenario_counts:
        if n > len(pv_liabilities):
            continue
        
        # Sample n scenarios (use first n for reproducibility)
        sample = pv_liabilities[:n]
        cte = calculate_cte(sample, 0.70)
        
        results.append({
            'n_scenarios': n,
            'cte70': cte,
            'std_error': np.std(sample[int(n*0.7):]) / np.sqrt(int(n*0.3)),
        })
    
    return pd.DataFrame(results)

# Test different scenario counts
scenario_counts = [50, 100, 200, 300, 500, 750, 1000]
convergence_df = convergence_analysis(pv_liabilities, scenario_counts)

# Plot convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# CTE70 convergence
ax1.plot(convergence_df['n_scenarios'], convergence_df['cte70'], 'b-o', linewidth=2, markersize=8)
ax1.axhline(cte70, color='red', linestyle='--', label=f'Final CTE70: ${cte70:,.0f}')
ax1.fill_between(
    convergence_df['n_scenarios'],
    convergence_df['cte70'] - 2*convergence_df['std_error'],
    convergence_df['cte70'] + 2*convergence_df['std_error'],
    alpha=0.2, label='±2 SE'
)
ax1.set_xlabel('Number of Scenarios')
ax1.set_ylabel('CTE70 ($)')
ax1.set_title('CTE70 Convergence')
ax1.legend()

# Standard error
ax2.plot(convergence_df['n_scenarios'], convergence_df['std_error'], 'g-o', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Scenarios')
ax2.set_ylabel('Standard Error ($)')
ax2.set_title('Estimation Uncertainty')

plt.tight_layout()
plt.show()

print("\nConvergence Table:")
print(convergence_df.to_string(index=False))

## 8. Sensitivity Analysis

How does CTE70 change with different market parameters?

In [ ]:
def run_sensitivity(param_name: str, param_values: List[float], base_params: dict) -> pd.DataFrame:
    """
    Run sensitivity analysis for a single parameter.
    """
    results = []
    
    for val in param_values:
        params = base_params.copy()
        params[param_name] = val
        
        # Generate paths with modified parameter
        paths = generate_equity_paths(
            500, N_YEARS, STEPS_PER_YEAR,
            params['drift'], params['volatility']
        )
        
        # Calculate liabilities
        liab = project_glwb_liability(
            paths, ACCOUNT_VALUE, BENEFIT_BASE,
            params['withdrawal_rate'], RIDER_FEE, FUND_FEE,
            params['risk_free_rate'], STEPS_PER_YEAR
        )
        
        cte = calculate_cte(liab, 0.70)
        results.append({param_name: val, 'cte70': cte})
    
    return pd.DataFrame(results)

# Base parameters
base_params = {
    'drift': EQUITY_DRIFT,
    'volatility': EQUITY_VOL,
    'risk_free_rate': RISK_FREE_RATE,
    'withdrawal_rate': WITHDRAWAL_RATE,
}

# Run sensitivity analyses
print("Running sensitivity analysis (this may take a moment)...")

vol_sens = run_sensitivity('volatility', [0.12, 0.15, 0.18, 0.21, 0.24, 0.27], base_params)
rate_sens = run_sensitivity('risk_free_rate', [0.02, 0.03, 0.04, 0.05, 0.06], base_params)
wd_sens = run_sensitivity('withdrawal_rate', [0.03, 0.04, 0.05, 0.06, 0.07], base_params)

print("Done!")

In [ ]:
# Plot sensitivity results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Volatility sensitivity
axes[0].plot(vol_sens['volatility'] * 100, vol_sens['cte70'] / 1000, 'b-o', linewidth=2, markersize=8)
axes[0].axvline(EQUITY_VOL * 100, color='red', linestyle='--', alpha=0.5, label='Base case')
axes[0].set_xlabel('Volatility (%)')
axes[0].set_ylabel('CTE70 ($000s)')
axes[0].set_title('CTE70 vs Volatility')
axes[0].legend()

# Interest rate sensitivity
axes[1].plot(rate_sens['risk_free_rate'] * 100, rate_sens['cte70'] / 1000, 'g-o', linewidth=2, markersize=8)
axes[1].axvline(RISK_FREE_RATE * 100, color='red', linestyle='--', alpha=0.5, label='Base case')
axes[1].set_xlabel('Risk-Free Rate (%)')
axes[1].set_ylabel('CTE70 ($000s)')
axes[1].set_title('CTE70 vs Interest Rate')
axes[1].legend()

# Withdrawal rate sensitivity
axes[2].plot(wd_sens['withdrawal_rate'] * 100, wd_sens['cte70'] / 1000, 'r-o', linewidth=2, markersize=8)
axes[2].axvline(WITHDRAWAL_RATE * 100, color='red', linestyle='--', alpha=0.5, label='Base case')
axes[2].set_xlabel('Withdrawal Rate (%)')
axes[2].set_ylabel('CTE70 ($000s)')
axes[2].set_title('CTE70 vs Withdrawal Rate')
axes[2].legend()

plt.tight_layout()
plt.show()

## 9. Key Takeaways

### CTE70 Reserve Characteristics

1. **CTE70 > Mean**: Reserve captures tail risk, not average outcomes
2. **Convergence**: ~500+ scenarios needed for stable estimates
3. **Volatility sensitivity**: Higher vol → significantly higher reserves
4. **Rate sensitivity**: Higher rates → lower reserves (discounting effect)
5. **Withdrawal sensitivity**: Higher withdrawals → higher guarantee cost

### Regulatory Context

- VM-21 requires **stochastic reserves** using CTE70
- Insurers must run 1,000+ scenarios
- Scenarios must be risk-neutral for pricing consistency
- Annual updates required for changing market conditions

### InsuranceAI Toolkit

The **InsuranceAI Toolkit** automates this entire process:
- Real-time scenario generation
- Automatic CTE70 calculation
- Sensitivity analysis dashboard
- Integration with hedging workflows

Try it: [InsuranceAI Toolkit Demo](https://insurance-ai-toolkit.streamlit.app)

---

## References

1. NAIC Valuation Manual (VM-21): Requirements for Principle-Based Reserves for Variable Annuities
2. Society of Actuaries: Variable Annuity Guaranteed Living Benefits
3. Hardy, M. (2003): Investment Guarantees: Modeling and Risk Management for Equity-Linked Life Insurance

---

*Created by Brandon Behring | [LinkedIn](https://www.linkedin.com/in/brandon-behring/) | [InsuranceAI Toolkit](https://github.com/brandonmbehring-dev/insurance-ai-toolkit)*